# K Nearest Neighbors Classifier

**Basic steps:**

1. Import the learning algorithm
2. Instantiate the model (choose hyper-parameters)
3. Learn the model
4. Predict the response

In [1]:
import pandas as pd


# Get Example Data

- target = InMichelin, whether or not a restaurant is in the Michelin guide
- Michelin Rating for restaurant in NYC
- encoding: default is UTF-8, this should specify latin_1

In [2]:
data = pd.read_csv("http://gattonweb.uky.edu/sheather/book/docs/datasets/MichelinNY.csv" , encoding="latin_1")
# y = InMichelin
data.head()

,InMichelin,Restaurant Name,Food,Decor,Service,Price
0,0,14 Wall Street,19,20,19,50
1,0,212,17,17,16,43
2,0,26 Seats,23,17,21,35
3,1,44,19,23,16,52
4,0,A,23,12,19,24


In [3]:
# Slice data 
# Delete extra variable that is not continuous

# exclude string column 'restaurant name'

data = data.loc[:, data.columns != 'Restaurant Name']

data.head()

,InMichelin,Food,Decor,Service,Price
0,0,19,20,19,50
1,0,17,17,16,43
2,0,23,17,21,35
3,1,19,23,16,52
4,0,23,12,19,24


# Change variable names to X, y to create train/test split

In [4]:
# define DV (y) and IV (x)
y = data['InMichelin']

# slice for x variables, KNN used for continuous X
# shold learn X well in real life
X = data.loc[:, data.columns != 'InMichelin']

# first five rows
# should specify print
print(y[0:5])

# first five rows
X.head()


0    0
1    0
2    0
3    1
4    0
Name: InMichelin, dtype: int64


,Food,Decor,Service,Price
0,19,20,19,50
1,17,17,16,43
2,23,17,21,35
3,19,23,16,52
4,23,12,19,24


# Train test split

In [5]:
# use 
from sklearn.model_selection import train_test_split

# Use train_test_split(X,y) to create four new data sets, defaults to .75/.25 split
X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train.head()

,Food,Decor,Service,Price
106,25,19,23,50
60,21,26,21,58
17,17,24,17,52
74,22,20,23,50
28,24,22,24,66


### Train model with k=5

In [6]:
from sklearn.neighbors import KNeighborsClassifier

# initialize
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

#Print accuracy rounded to two digits to the right of decimal
print("accuracy: {:.2f}".format(knn.score(X_test, y_test)))

# store predict test values in a list
y_pred = knn.predict(X_test) # y_pred includes your predictions

accuracy: 0.80


### Train model with k=10

In [7]:
# initialize
knn = KNeighborsClassifier(n_neighbors=10)

# fit
knn.fit(X_train, y_train)

# show precision
#Print accuracy rounded to two digits to the right of decimal
print("accuracy: {:.2f}".format(knn.score(X_test, y_test)))
y_pred = knn.predict(X_test)


accuracy: 0.83


In [8]:
y_pred # view predictions for test data

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0])

## Using Cross validation for model evaluation

how we envaluate our model use cross validation

In [1]:
# import cross validation functions from sk learn

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import RepeatedKFold

from sklearn.model_selection import StratifiedKFold


/anaconda/envs/ml/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence
/anaconda/envs/ml/lib/python3.7/site-packages/sklearn/model_selection/_split.py:18: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


In [2]:
#
# change fold num
from sklearn.model_selection import KFold

# Set up function parameters for diff't cross validation strategies


kfold = KFold(n_splits=5)
# shuffle the data
skfold = StratifiedKFold(n_splits=5, shuffle=True)

# repeat 
rkf = RepeatedKFold(n_splits=5, n_repeats=10)

# use KFold validation
print("KFold:\n{}".format(
cross_val_score(KNeighborsClassifier(), X, y, cv=kfold)))

print("StratifiedKFold:\n{}".format(
cross_val_score(KNeighborsClassifier(n_neighbors=5), X, y, cv=skfold)))

print("RepeatedKFold:\n{}".format(
cross_val_score(KNeighborsClassifier(n_neighbors=5), X, y, cv=rkf)))


# usually calculate the average precsion for 

NameError: name 'KNeighborsClassifier' is not defined

## Tuning models with grid search

In [20]:
# same idea, more automatic
from sklearn.model_selection import GridSearchCV
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

#create dictionary data object with keys equal to parameter name 'n_neighbors' 
#for knn model and values equal to range of k values to create models for

#set a series of hpyer parameter
param_grid = {'n_neighbors': np.arange(1, 15, 2)} #np.arange creates sequence of numbers for each k value


grid = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid, cv=10)
    
#use meta model methods to fit score and predict model:
grid.fit(X_train, y_train)

#extract best score and parameter by calling objects "best_score_" and "best_params_"
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))
print("test-set score: {:.3f}".format(grid.score(X_test, y_test)))


best mean cross-validation score: 0.772
best parameters: {'n_neighbors': 13}
test-set score: 0.927


In [74]:
# view data with complete tuning results
results = pd.DataFrame(grid.cv_results_)
results


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_n_neighbors,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split7_test_score,split7_train_score,split8_test_score,split8_train_score,split9_test_score,split9_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.0009,0.0005,0.682927,1.000000,1,{u'n_neighbors': 1},7,1.000000,1.000000,0.769231,...,0.416667,1.000000,0.636364,1.000000,0.909091,1.000000,0.000539,0.000500,0.182205,0.000000
1,0.0009,0.0005,0.739837,0.847301,3,{u'n_neighbors': 3},6,0.692308,0.827273,0.846154,...,0.583333,0.873874,0.818182,0.857143,0.727273,0.848214,0.000300,0.000500,0.124125,0.015701
2,0.0008,0.0005,0.764228,0.841911,5,{u'n_neighbors': 5},5,0.846154,0.818182,0.923077,...,0.500000,0.873874,0.909091,0.812500,0.818182,0.857143,0.000400,0.000500,0.134918,0.019855
3,0.0007,0.0006,0.821138,0.822042,7,{u'n_neighbors': 7},1,1.000000,0.809091,0.846154,...,0.666667,0.846847,0.909091,0.812500,0.818182,0.821429,0.000458,0.000490,0.110445,0.011399
4,0.0007,0.0003,0.796748,0.812123,9,{u'n_neighbors': 9},2,1.000000,0.781818,0.846154,...,0.583333,0.846847,0.909091,0.794643,0.818182,0.803571,0.000458,0.000458,0.119982,0.018205
5,0.0006,0.0006,0.796748,0.803081,11,{u'n_neighbors': 11},2,1.000000,0.772727,0.846154,...,0.583333,0.828829,0.909091,0.794643,0.818182,0.794643,0.000490,0.000490,0.119982,0.015396
6,0.0008,0.0004,0.780488,0.803982,13,{u'n_neighbors': 13},4,1.000000,0.772727,0.846154,...,0.500000,0.855856,0.818182,0.785714,0.818182,0.803571,0.000400,0.000490,0.131116,0.023481
